In [76]:
!pip install audiomentations

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [77]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from scipy.io import wavfile
import os.path
import IPython.display
import seaborn as sns
import librosa
import librosa.display
import soundfile
from audiomentations import Compose, AddGaussianNoise, TimeStretch, PitchShift

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import confusion_matrix

import tensorflow as tf
from tensorflow.keras import utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Conv1D, MaxPooling1D, Flatten, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras import optimizers

from keras import optimizers,regularizers
from tensorflow.python.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.decomposition import PCA
import warnings

In [2]:
!pip install pydub
!pip install audiomentations

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [78]:
import re
import os
import wave
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pydub import AudioSegment
from IPython.display import Audio, display
import librosa as lib
import librosa.display

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [79]:
PATH = "/content/drive/MyDrive/Colab Notebooks/Crema"
AUDIO_PATH = "/content/drive/MyDrive/Colab Notebooks/Crema/1001_DFA_ANG_XX.wav"

In [80]:
def zeroCrossingRate(audio):
  return lib.feature.zero_crossing_rate(audio)

In [81]:
def energy(audio):
  # using a spectrogram will give a more accurate representation
  # of energy over time because its frames can be windowed
  S, phase = lib.magphase(lib.stft(audio))
  return lib.feature.rms(S=S).mean()

In [8]:
def chromaStft(audio, sr):
  stft = np.abs(librosa.stft(audio))
  return librosa.feature.chroma_stft(S=stft, sr=sr, n_fft=200).mean()

In [9]:
def mfcc(data, sr):
  return librosa.feature.mfcc(y=data, sr=sr, n_fft=200).mean()

In [10]:
def melSpectrogram(audio, sr):
  mel_spectrogram = lib.feature.melspectrogram(y=audio, sr=sr, n_fft=200, n_mels=64)
  log_mel_spectrogram = librosa.power_to_db(mel_spectrogram)
  return log_mel_spectrogram

In [11]:
def playAudio(audio_file):
  audio = AudioSegment.from_wav(audio_file)
  # Play the audio
  audio.export('temp_audio.wav', format='wav')
  audio_data = open('temp_audio.wav', 'rb').read()
  display(Audio(audio_data))
  # Delete the temporary audio file
  os.remove('temp_audio.wav')

In [12]:
def visualize_waveform(audio, sr):
    plt.figure(figsize=(12, 4))
    plt.plot(audio)
    plt.xlabel('Time')
    plt.ylabel('Amplitude')
    plt.title('Waveform')
    plt.show()

In [82]:
def splitData(dataset_path):
  audio_files = glob.glob(os.path.join(dataset_path, "*.wav"))
  audio_train, audio_test = train_test_split(audio_files, test_size=0.3, random_state=42, shuffle=True)
  return audio_train, audio_test

In [83]:
D_train, D_test = splitData(PATH)

In [84]:
print(len(D_train), len(D_test))

5216 2236


In [85]:
D_test[0]

'/content/drive/MyDrive/Colab Notebooks/Crema/1017_IEO_SAD_HI.wav'

In [86]:
def getMaxLen(audio_files):
  max_length = 0
  for audio_file in audio_files:
    audio, _ = librosa.load(audio_file)
    length = len(audio)
    max_length = max(max_length, length)
  return max_length

In [97]:
def loadAudioTime(audio_files):
  classes = ["SAD", "ANG", "DIS", "FEA", "HAP", "NEU"]
  
  # max_length = getMaxLen(audio_files)
  max_length = 200
  D, Y = [], []
  for audio_file in audio_files: 
    # load the audio file
    audio, sr = lib.load(audio_file, sr=4444)
    # extract zero crossing rate
    zcr = zeroCrossingRate(audio)
    # extract energy
    rms = energy(audio)
    # chroma stft
    cs = chromaStft(audio, sr)
    # mfcc
    mfc = mfcc(audio, sr)
    # extract mel spectrogram
    #mel_spec = melSpectrogram(audio, sr).mean()
    combined_features = np.concatenate(([rms, cs, mfc], (np.pad(zcr[0], (0, 300 - len(zcr[0]))))))
    D.append(combined_features)
    for cls in classes:
      if re.search(cls, audio_file): Y.append(cls)
    
  return D, Y

In [96]:
def loadAudioFreq(audio_files):
  classes = ["SAD", "ANG", "DIS", "FEA", "HAP", "NEU"]
 
  # max_length = getMaxLen(audio_files)
  max_length = 200
  D, Y = [], []
  for audio_file in audio_files: 
    # load the audio file
    audio, sr = lib.load(audio_file, sr=4444)
    # extract mel spectrogram
    mel_spec = melSpectrogram(audio, sr)
    D.append(mel_spec)
    for cls in classes:
      if re.search(cls, audio_file): Y.append(cls)
   
  return D, Y

## Time Domain Model

In [98]:
D_time, Y_time = loadAudioTime(D_train)

In [99]:
D_time = np.array(D_time)
D_time.shape

(5216, 303)

In [100]:
unicheck = np.unique(Y_time, axis=0)
print(f"Check on time domain:\n{unicheck}")

Check on time domain:
['ANG' 'DIS' 'FEA' 'HAP' 'NEU' 'SAD']


In [101]:
df_time = pd.DataFrame(Y_time)
df_time.head()

,0
0,SAD
1,FEA
2,SAD
3,ANG
4,SAD


In [102]:
# Use the get_dummies() method to one-hot encode the labels
one_hot_labels = pd.get_dummies(df_time)

# Convert the DataFrame to a NumPy array
one_hot_labels = one_hot_labels.to_numpy()

print(one_hot_labels.shape)

(5216, 6)


In [103]:
Y_time = one_hot_labels
Y_time.shape

(5216, 6)

In [104]:
# Apply PCA to the data
pca = PCA(n_components=100)
D_pca = pca.fit_transform(D_time)

In [105]:
# Split the data into train and test sets
x_train, x_test, y_train, y_test = train_test_split(np.array(D_pca), np.array(Y_time), test_size=0.05, random_state=69, shuffle=True, stratify=Y_time)
print((x_train.shape, y_train.shape, x_test.shape, y_test.shape))

x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))

model = Sequential()
model.add(Conv1D(128, 3, input_shape=(x_train.shape[1], 1), kernel_regularizer=regularizers.l2(0.001)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(256, 3, kernel_regularizer=regularizers.l2(0.001)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(512, 3, kernel_regularizer=regularizers.l2(0.001)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(1024, 3,  kernel_regularizer=regularizers.l2(0.001)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Flatten())
model.add(Dense(512, kernel_regularizer=regularizers.l2(0.001)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.3))

model.add(Dense(256, kernel_regularizer=regularizers.l2(0.001)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(128, kernel_regularizer=regularizers.l2(0.001)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(6, activation='softmax'))

model.compile(loss='categorical_crossentropy',
             optimizer=optimizers.Adam(lr=0.0005),
             metrics=['accuracy'])

# Specify thecallbacks for the model
callbacks = [EarlyStopping(monitor='val_loss', patience=10)]

model.summary()

# Train the model with early stopping
history = model.fit(x_train, y_train,
                    validation_data=(x_train, y_train),
                    epochs=50,
                    batch_size=32,
                    callbacks=callbacks)

((4955, 100), (4955, 6), (261, 100), (261, 6))
Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_12 (Conv1D)          (None, 98, 128)           512       
                                                                 
 batch_normalization_21 (Bat  (None, 98, 128)          512       
 chNormalization)                                                
                                                                 
 activation_21 (Activation)  (None, 98, 128)           0         
                                                                 
 max_pooling1d_12 (MaxPoolin  (None, 49, 128)          0         
 g1D)                                                            
                                                                 
 conv1d_13 (Conv1D)          (None, 47, 256)           98560     
                                                                 
 batch_

/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/50
155/155 [==============================] - 5s 18ms/step - loss: 3.7800 - accuracy: 0.3005 - val_loss: 3.8594 - val_accuracy: 0.1483
Epoch 2/50
155/155 [==============================] - 2s 14ms/step - loss: 3.1293 - accuracy: 0.3395 - val_loss: 3.5364 - val_accuracy: 0.1483
Epoch 3/50
155/155 [==============================] - 2s 15ms/step - loss: 2.8142 - accuracy: 0.3703 - val_loss: 3.1361 - val_accuracy: 0.1546
Epoch 4/50
155/155 [==============================] - 3s 23ms/step - loss: 2.6385 - accuracy: 0.3707 - val_loss: 2.5732 - val_accuracy: 0.3443
Epoch 5/50
155/155 [==============================] - 2s 15ms/step - loss: 2.4932 - accuracy: 0.3980 - val_loss: 2.5006 - val_accuracy: 0.3631
Epoch 6/50
155/155 [==============================] - 2s 14ms/step - loss: 2.3657 - accuracy: 0.4117 - val_loss: 2.3715 - val_accuracy: 0.4117
Epoch 7/50
155/155 [==============================] - 2s 14ms/step - loss: 2.2999 - accuracy: 0.4085 - val_loss: 2.3271 - val_accuracy: 0.3778

## Frequency Domain Model

In [106]:
D_freq, Y_freq = loadAudioFreq(D_train)

In [107]:
unicheck = np.unique(Y_freq, axis=0)
print(f"Check on freq domain:\n{unicheck}")

Check on freq domain:
['ANG' 'DIS' 'FEA' 'HAP' 'NEU' 'SAD']


In [108]:
df_freq = pd.DataFrame(Y_freq)
df_freq.head()

,0
0,SAD
1,FEA
2,SAD
3,ANG
4,SAD


In [109]:
# Use the get_dummies() method to one-hot encode the labels
one_hot_labels = pd.get_dummies(df_freq)

# Convert the DataFrame to a NumPy array
one_hot_labels = one_hot_labels.to_numpy()

print(one_hot_labels.shape)

(5216, 6)


In [110]:
Y_freq = one_hot_labels
Y_freq.shape

(5216, 6)

In [111]:
for i, array in enumerate(D_freq):
    D_freq[i] = np.pad(array, ((0, 0), (0, 256 - array.shape[1])))
D_freq = np.array(D_freq)

In [112]:
D_freq.shape

(5216, 64, 256)

In [113]:
x_train, x_test, y_train, y_test = train_test_split(np.array(D_freq), np.array(Y_freq), test_size=0.05)
print((x_train.shape, y_train.shape, x_test.shape, y_test.shape))

((4955, 64, 256), (4955, 6), (261, 64, 256), (261, 6))


In [114]:
x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], x_train.shape[2], 1))
x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], x_test.shape[2], 1))
print(x_train.shape, x_test.shape)

(4955, 64, 256, 1) (261, 64, 256, 1)


In [115]:
model = Sequential()

# Layer 1: Convolutional Layer
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(x_train.shape[1], x_train.shape[2], 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Layer 2: Convolutional Layer
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Layer 3: Convolutional Layer
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Layer 4: Flatten Layer
model.add(Flatten())

# Layer 5: Fully Connected Layer
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))

# Layer 6: Fully Connected Layer
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

# Layer 7: Output Layer
model.add(Dense(6, activation='sigmoid'))

model.compile(loss='categorical_crossentropy',
             optimizer=optimizers.Adam(lr=0.0005),
             metrics=['accuracy'])

model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 62, 254, 32)       320       
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 31, 127, 32)      0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 29, 125, 64)       18496     
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 14, 62, 64)       0         
 2D)                                                             
                                                                 
 conv2d_8 (Conv2D)           (None, 12, 60, 128)       73856     
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 6, 30, 128)      

/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [116]:
# Specify thecallbacks for the model
callbacks = [EarlyStopping(monitor='val_loss', patience=10)]

history = model.fit(x_train, y_train,
                    epochs=50,
                    batch_size=128,
                    validation_data=(x_test, y_test),
                    callbacks=callbacks)

Epoch 1/50
39/39 [==============================] - 3s 62ms/step - loss: 1.9521 - accuracy: 0.1909 - val_loss: 1.7365 - val_accuracy: 0.2950
Epoch 2/50
39/39 [==============================] - 2s 50ms/step - loss: 1.6991 - accuracy: 0.2700 - val_loss: 1.5977 - val_accuracy: 0.3295
Epoch 3/50
39/39 [==============================] - 2s 50ms/step - loss: 1.6058 - accuracy: 0.3150 - val_loss: 1.5118 - val_accuracy: 0.3755
Epoch 4/50
39/39 [==============================] - 2s 50ms/step - loss: 1.5678 - accuracy: 0.3310 - val_loss: 1.4767 - val_accuracy: 0.3563
Epoch 5/50
39/39 [==============================] - 2s 50ms/step - loss: 1.5304 - accuracy: 0.3623 - val_loss: 1.4488 - val_accuracy: 0.3870
Epoch 6/50
39/39 [==============================] - 2s 50ms/step - loss: 1.5156 - accuracy: 0.3752 - val_loss: 1.4339 - val_accuracy: 0.3908
Epoch 7/50
39/39 [==============================] - 2s 53ms/step - loss: 1.4981 - accuracy: 0.3776 - val_loss: 1.4381 - val_accuracy: 0.4061
Epoch 8/50
39

In [117]:
def loadAudioTime(audio_files):
  classes = ["SAD", "ANG", "DIS", "FEA", "HAP", "NEU"]
 
  # max_length = getMaxLen(audio_files)
  max_length = 200
  D, Y = [], []
  for audio_file in audio_files: 
    # load the audio file
    audio, sr = lib.load(audio_file, sr=4444)
    # extract zero crossing rate
    zcr = zeroCrossingRate(audio)
    # extract energy
    rms = energy(audio)
    # chroma stft
    cs = chromaStft(audio, sr)
    # mfcc
    mfc = mfcc(audio, sr)
    # extract mel spectrogram
    #mel_spec = melSpectrogram(audio, sr).mean()
    combined_features = np.concatenate(([rms, cs, mfc], (np.pad(zcr[0], (0, 300 - len(zcr[0]))))))
    D.append(combined_features)
    for cls in classes:
      if re.search(cls, audio_file): Y.append(cls)

  return D, Y

In [118]:
D_time, Y_time = loadAudioTime(D_train)

/usr/local/lib/python3.10/dist-packages/librosa/util/decorators.py:88: UserWarning: Trying to estimate tuning from empty frequency set.
  return f(*args, **kwargs)


In [119]:
D_time = np.array(D_time)
D_time.shape

(5216, 303)

In [120]:
unicheck = np.unique(Y_time, axis=0)
print(f"Check on time domain:\n{unicheck}")

Check on time domain:
['ANG' 'DIS' 'FEA' 'HAP' 'NEU' 'SAD']


In [121]:
df_time = pd.DataFrame(Y_time)
df_time.head()

,0
0,SAD
1,FEA
2,SAD
3,ANG
4,SAD


In [122]:
# Use the get_dummies() method to one-hot encode the labels
one_hot_labels = pd.get_dummies(df_time)

# Convert the DataFrame to a NumPy array
one_hot_labels = one_hot_labels.to_numpy()

print(one_hot_labels.shape)

(5216, 6)


In [123]:
Y_time = one_hot_labels
Y_time.shape

(5216, 6)

In [124]:
# Apply PCA to the data
pca = PCA(n_components=100)
D_pca = pca.fit_transform(D_time)

In [125]:
# Split the data into train and test sets
x_train, x_test, y_train, y_test = train_test_split(np.array(D_pca), np.array(Y_time), test_size=0.05, random_state=69, shuffle=True, stratify=Y_time)
print((x_train.shape, y_train.shape, x_test.shape, y_test.shape))

x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))

model = Sequential()
model.add(Conv1D(128, 3, input_shape=(x_train.shape[1], 1), kernel_regularizer=regularizers.l2(0.001)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(256, 3, kernel_regularizer=regularizers.l2(0.001)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(512, 3, kernel_regularizer=regularizers.l2(0.001)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(1024, 3,  kernel_regularizer=regularizers.l2(0.001)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Flatten())
model.add(Dense(512, kernel_regularizer=regularizers.l2(0.001)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.3))

model.add(Dense(256, kernel_regularizer=regularizers.l2(0.001)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(128, kernel_regularizer=regularizers.l2(0.001)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(6, activation='softmax'))

model.compile(loss='categorical_crossentropy',
             optimizer=optimizers.Adam(lr=0.0005),
             metrics=['accuracy'])

# Specify thecallbacks for the model
callbacks = [EarlyStopping(monitor='val_loss', patience=10)]

model.summary()

# Train the model with early stopping
history = model.fit(x_train, y_train,
                    validation_data=(x_train, y_train),
                    epochs=50,
                    batch_size=32,
                    callbacks=callbacks)

((4955, 100), (4955, 6), (261, 100), (261, 6))
Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_16 (Conv1D)          (None, 98, 128)           512       
                                                                 
 batch_normalization_28 (Bat  (None, 98, 128)          512       
 chNormalization)                                                
                                                                 
 activation_28 (Activation)  (None, 98, 128)           0         
                                                                 
 max_pooling1d_16 (MaxPoolin  (None, 49, 128)          0         
 g1D)                                                            
                                                                 
 conv1d_17 (Conv1D)          (None, 47, 256)           98560     
                                                                 
 batch_

/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/50
155/155 [==============================] - 5s 16ms/step - loss: 3.8052 - accuracy: 0.2993 - val_loss: 4.0004 - val_accuracy: 0.1485
Epoch 2/50
155/155 [==============================] - 2s 14ms/step - loss: 3.1712 - accuracy: 0.3393 - val_loss: 3.5646 - val_accuracy: 0.1574
Epoch 3/50
155/155 [==============================] - 3s 19ms/step - loss: 2.8701 - accuracy: 0.3540 - val_loss: 3.2397 - val_accuracy: 0.1679
Epoch 4/50
155/155 [==============================] - 3s 18ms/step - loss: 2.6665 - accuracy: 0.3721 - val_loss: 2.6759 - val_accuracy: 0.3243
Epoch 5/50
155/155 [==============================] - 2s 14ms/step - loss: 2.5222 - accuracy: 0.3851 - val_loss: 2.4630 - val_accuracy: 0.3715
Epoch 6/50
155/155 [==============================] - 2s 14ms/step - loss: 2.3881 - accuracy: 0.3984 - val_loss: 2.3954 - val_accuracy: 0.3764
Epoch 7/50
155/155 [==============================] - 2s 15ms/step - loss: 2.2922 - accuracy: 0.4065 - val_loss: 2.2504 - val_accuracy: 0.4109

**Without Augmentation**

In [126]:
def getMaxLen(audio_files):
  max_length = 0
  for audio_file in audio_files:
    audio, _ = librosa.load(audio_file)
    length = len(audio)
    max_length = max(max_length, length)
  return max_length

In [127]:
def loadAudioTime(audio_files):
  classes = ["SAD", "ANG", "DIS", "FEA", "HAP", "NEU"]
  #1D WITHOUT MEL
  # max_length = getMaxLen(audio_files)
  max_length = 200
  D, Y = [], []
  for audio_file in audio_files: 
    # load the audio file
    audio, sr = lib.load(audio_file, sr=4444)
    # extract zero crossing rate
    zcr = zeroCrossingRate(audio)
    # extract energy
    rms = energy(audio)
    # chroma stft
    cs = chromaStft(audio, sr)
    # mfcc
    mfc = mfcc(audio, sr)
    # extract mel spectrogram
    #mel_spec = melSpectrogram(audio, sr).mean()
    combined_features = np.concatenate(([rms, cs, mfc], (np.pad(zcr[0], (0, 300 - len(zcr[0]))))))
    D.append(combined_features)
    for cls in classes:
      if re.search(cls, audio_file): Y.append(cls)
  return D,Y
  

In [128]:
def loadAudioFreq(audio_files):
  classes = ["SAD", "ANG", "DIS", "FEA", "HAP", "NEU"]
  
  # max_length = getMaxLen(audio_files)
  max_length = 200
  D, Y = [], []
  for audio_file in audio_files: 
    # load the audio file
    audio, sr = lib.load(audio_file, sr=4444)
    # extract mel spectrogram
    mel_spec = melSpectrogram(audio, sr)
    D.append(mel_spec)
    for cls in classes:
      if re.search(cls, audio_file): Y.append(cls)
    
  return D, Y

In [129]:
D_time, Y_time = loadAudioTime(D_train)

/usr/local/lib/python3.10/dist-packages/librosa/util/decorators.py:88: UserWarning: Trying to estimate tuning from empty frequency set.
  return f(*args, **kwargs)


In [130]:
D_time = np.array(D_time)
D_time.shape

(5216, 303)

In [131]:
unicheck = np.unique(Y_time, axis=0)
print(f"Check on time domain:\n{unicheck}")

Check on time domain:
['ANG' 'DIS' 'FEA' 'HAP' 'NEU' 'SAD']


In [132]:
df_time = pd.DataFrame(Y_time)
df_time.head()

,0
0,SAD
1,FEA
2,SAD
3,ANG
4,SAD


In [133]:
# Use the get_dummies() method to one-hot encode the labels
one_hot_labels = pd.get_dummies(df_time)

# Convert the DataFrame to a NumPy array
one_hot_labels = one_hot_labels.to_numpy()

print(one_hot_labels.shape)

(5216, 6)


In [134]:
Y_time = one_hot_labels
Y_time.shape

(5216, 6)

In [135]:
# Apply PCA to the data
pca = PCA(n_components=150)
D_pca = pca.fit_transform(D_time)

In [136]:
# Split the data into train and test sets
x_train, x_test, y_train, y_test = train_test_split(np.array(D_pca), np.array(Y_time), test_size=0.05, random_state=69, shuffle=True, stratify=Y_time)
print((x_train.shape, y_train.shape, x_test.shape, y_test.shape))

x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))

model = Sequential()
model.add(Conv1D(128, 3, input_shape=(x_train.shape[1], 1), kernel_regularizer=regularizers.l2(0.001)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(256, 3, kernel_regularizer=regularizers.l2(0.001)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(512, 3, kernel_regularizer=regularizers.l2(0.001)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(1024, 3,  kernel_regularizer=regularizers.l2(0.001)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Flatten())
model.add(Dense(512, kernel_regularizer=regularizers.l2(0.001)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.3))

model.add(Dense(256, kernel_regularizer=regularizers.l2(0.001)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(128, kernel_regularizer=regularizers.l2(0.001)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(6, activation='softmax'))

model.compile(loss='categorical_crossentropy',
             optimizer=optimizers.Adam(lr=0.0005),
             metrics=['accuracy'])

# Specify thecallbacks for the model
callbacks = [EarlyStopping(monitor='val_loss', patience=10)]

model.summary()

# Train the model with early stopping
history = model.fit(x_train, y_train,
                    validation_data=(x_train, y_train),
                    epochs=50,
                    batch_size=32,
                    callbacks=callbacks)

((4955, 150), (4955, 6), (261, 150), (261, 6))
Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_20 (Conv1D)          (None, 148, 128)          512       
                                                                 
 batch_normalization_35 (Bat  (None, 148, 128)         512       
 chNormalization)                                                
                                                                 
 activation_35 (Activation)  (None, 148, 128)          0         
                                                                 
 max_pooling1d_20 (MaxPoolin  (None, 74, 128)          0         
 g1D)                                                            
                                                                 
 conv1d_21 (Conv1D)          (None, 72, 256)           98560     
                                                                 
 batch_

/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/50
155/155 [==============================] - 7s 24ms/step - loss: 3.5932 - accuracy: 0.2971 - val_loss: 3.7270 - val_accuracy: 0.1647
Epoch 2/50
155/155 [==============================] - 2s 15ms/step - loss: 2.9795 - accuracy: 0.3439 - val_loss: 3.5653 - val_accuracy: 0.1485
Epoch 3/50
155/155 [==============================] - 2s 16ms/step - loss: 2.6992 - accuracy: 0.3572 - val_loss: 2.9821 - val_accuracy: 0.1639
Epoch 4/50
155/155 [==============================] - 3s 19ms/step - loss: 2.5389 - accuracy: 0.3631 - val_loss: 2.5671 - val_accuracy: 0.3082
Epoch 5/50
155/155 [==============================] - 3s 18ms/step - loss: 2.4151 - accuracy: 0.3707 - val_loss: 2.3263 - val_accuracy: 0.3933
Epoch 6/50
155/155 [==============================] - 2s 16ms/step - loss: 2.3115 - accuracy: 0.3901 - val_loss: 2.2436 - val_accuracy: 0.3974
Epoch 7/50
155/155 [==============================] - 2s 16ms/step - loss: 2.2428 - accuracy: 0.3937 - val_loss: 2.3812 - val_accuracy: 0.3384

In [137]:
D_freq, Y_freq = loadAudioFreq(D_train)

In [138]:
unicheck = np.unique(Y_freq, axis=0)
print(f"Check on freq domain:\n{unicheck}")

Check on freq domain:
['ANG' 'DIS' 'FEA' 'HAP' 'NEU' 'SAD']


In [139]:
df_freq = pd.DataFrame(Y_freq)
df_freq.head()

,0
0,SAD
1,FEA
2,SAD
3,ANG
4,SAD


In [140]:
# Use the get_dummies() method to one-hot encode the labels
one_hot_labels = pd.get_dummies(df_freq)

# Convert the DataFrame to a NumPy array
one_hot_labels = one_hot_labels.to_numpy()

print(one_hot_labels.shape)

(5216, 6)


In [141]:
Y_freq = one_hot_labels
Y_freq.shape

(5216, 6)

In [142]:
for i, array in enumerate(D_freq):
    D_freq[i] = np.pad(array, ((0, 0), (0, 256 - array.shape[1])))
D_freq = np.array(D_freq)

In [143]:
D_freq.shape

(5216, 64, 256)

In [144]:
x_train, x_test, y_train, y_test = train_test_split(np.array(D_freq), np.array(Y_freq), test_size=0.05)
print((x_train.shape, y_train.shape, x_test.shape, y_test.shape))

((4955, 64, 256), (4955, 6), (261, 64, 256), (261, 6))


In [145]:
x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], x_train.shape[2], 1))
x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], x_test.shape[2], 1))
print(x_train.shape, x_test.shape)

(4955, 64, 256, 1) (261, 64, 256, 1)


In [146]:
model = Sequential()

# Layer 1: Convolutional Layer
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(x_train.shape[1], x_train.shape[2], 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Layer 2: Convolutional Layer
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Layer 3: Convolutional Layer
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Layer 4: Flatten Layer
model.add(Flatten())

# Layer 5: Fully Connected Layer
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))

# Layer 6: Fully Connected Layer
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

# Layer 7: Output Layer
model.add(Dense(6, activation='sigmoid'))

model.compile(loss='categorical_crossentropy',
             optimizer=optimizers.Adam(lr=0.0005),
             metrics=['accuracy'])

model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 62, 254, 32)       320       
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 31, 127, 32)      0         
 2D)                                                             
                                                                 
 conv2d_10 (Conv2D)          (None, 29, 125, 64)       18496     
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 14, 62, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_11 (Conv2D)          (None, 12, 60, 128)       73856     
                                                                 
 max_pooling2d_11 (MaxPoolin  (None, 6, 30, 128)      

/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [147]:
# Specify thecallbacks for the model
callbacks = [EarlyStopping(monitor='val_loss', patience=10)]

history = model.fit(x_train, y_train,
                    epochs=50,
                    batch_size=128,
                    validation_data=(x_test, y_test),
                    callbacks=callbacks)

Epoch 1/50
39/39 [==============================] - 3s 57ms/step - loss: 1.8736 - accuracy: 0.2085 - val_loss: 1.6491 - val_accuracy: 0.3525
Epoch 2/50
39/39 [==============================] - 2s 51ms/step - loss: 1.6492 - accuracy: 0.3092 - val_loss: 1.5099 - val_accuracy: 0.3946
Epoch 3/50
39/39 [==============================] - 2s 54ms/step - loss: 1.5781 - accuracy: 0.3362 - val_loss: 1.4900 - val_accuracy: 0.3985
Epoch 4/50
39/39 [==============================] - 2s 51ms/step - loss: 1.5458 - accuracy: 0.3485 - val_loss: 1.4563 - val_accuracy: 0.3908
Epoch 5/50
39/39 [==============================] - 2s 50ms/step - loss: 1.5265 - accuracy: 0.3588 - val_loss: 1.4571 - val_accuracy: 0.4291
Epoch 6/50
39/39 [==============================] - 2s 52ms/step - loss: 1.5121 - accuracy: 0.3711 - val_loss: 1.4143 - val_accuracy: 0.4100
Epoch 7/50
39/39 [==============================] - 2s 51ms/step - loss: 1.4836 - accuracy: 0.3867 - val_loss: 1.4414 - val_accuracy: 0.4176
Epoch 8/50
39